<a href="https://colab.research.google.com/github/HisakaKoji/pytorch_advanced/blob/master/7-7_after7-6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd pytorch_advanced/7_nlp_sentiment_transformer

/content/pytorch_advanced/7_nlp_sentiment_transformer


In [0]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

In [0]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

In [0]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [0]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

# 読み込み
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=64)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}

In [6]:
from utils.transformer import TransformerClassification

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# ネットワークの初期化を定義


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)


# 訓練モードに設定
net.train()

# TransformerBlockモジュールを初期化実行
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)


print('ネットワーク設定完了')

ネットワーク設定完了


In [0]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [0]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書オブジェクト

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # mask作成
                    input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
                    input_mask = (inputs != input_pad)

                    # Transformerに入力
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [10]:

# 学習・検証を実行する 15分ほどかかります
num_epochs = 10
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-----start-------
Epoch 1/10 | train |  Loss: 0.5999 Acc: 0.6666
Epoch 1/10 |  val  |  Loss: 0.4331 Acc: 0.8018
Epoch 2/10 | train |  Loss: 0.4383 Acc: 0.8019
Epoch 2/10 |  val  |  Loss: 0.3932 Acc: 0.8208
Epoch 3/10 | train |  Loss: 0.4070 Acc: 0.8199
Epoch 3/10 |  val  |  Loss: 0.3781 Acc: 0.8320
Epoch 4/10 | train |  Loss: 0.3860 Acc: 0.8288
Epoch 4/10 |  val  |  Loss: 0.3583 Acc: 0.8434
Epoch 5/10 | train |  Loss: 0.3700 Acc: 0.8357
Epoch 5/10 |  val  |  Loss: 0.3491 Acc: 0.8478
Epoch 6/10 | train |  Loss: 0.3584 Acc: 0.8432
Epoch 6/10 |  val  |  Loss: 0.3446 Acc: 0.8516
Epoch 7/10 | train |  Loss: 0.3470 Acc: 0.8496
Epoch 7/10 |  val  |  Loss: 0.3372 Acc: 0.8538
Epoch 8/10 | train |  Loss: 0.3427 Acc: 0.8505
Epoch 8/10 |  val  |  Loss: 0.3339 Acc: 0.8568
Epoch 9/10 | train |  Loss: 0.3382 Acc: 0.8534
Epoch 9/10 |  val  |  Loss: 0.3522 Acc: 0.8544
Epoch 10/10 | train |  Loss: 0.3329 Acc: 0.8551
Epoch 10/10 |  val  |  Loss: 0.3425 Acc: 0.8558


In [11]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)

epoch_corrects = 0  # epochの正解数

for batch in (test_dl):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    
    # GPUが使えるならGPUにデータを送る
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # mask作成
        input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
        input_mask = (inputs != input_pad)

        # Transformerに入力
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # ラベルを予測

        # 結果の計算
        # 正解数の合計を更新
        epoch_corrects += torch.sum(preds == labels.data)

# 正解率
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(test_dl.dataset),epoch_acc))

テストデータ25000個での正解率：0.8447


In [0]:
# HTMLを作成する関数を実装


def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2, TEXT):
    "HTMLデータを作成する"

    # indexの結果を抽出
    sentence = batch.Text[0][index]  # 文章
    label = batch.Label[index]  # ラベル
    pred = preds[index]  # 予測
    print(sentence)
    # indexのAttentionを抽出と規格化
    attens1 = normlized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
    attens1 /= attens1.max()
    print(attens1)
    attens2 = normlized_weights_2[index, 0, :]  # 0番目の<cls>のAttention
    attens2 /= attens2.max()

    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)

    # 1段目のAttention
    html += '[TransformerBlockの1段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 2段目のAttention
    html += '[TransformerBlockの2段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html

In [13]:
from IPython.display import HTML

# Transformerで処理

# ミニバッチの用意
batch = next(iter(test_dl))

# GPUが使えるならGPUにデータを送る
inputs = batch.Text[0].to(device)  # 文章
labels = batch.Label.to(device)  # ラベル

# mask作成
input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (inputs != input_pad)

# Transformerに入力
outputs, normlized_weights_1, normlized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # ラベルを予測


index = 3  # 出力させたいデータ
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力

tensor([    2,   434,    10,     4,  4772,  8600,     9,  1774,  5766,   510,
         7764, 10174,   297,     6,  4847,     6,     0,     7,     8,   374,
            9,    88,  2433,  4651,     5,    44,    30,    41,   176,   105,
         4633,    20,   186,   440,     5,     5,     5,    23,    15,    34,
           11,     6,     7,    12,    17,    90,     5,   152,     4,   115,
           11,     8,   123,  5846,    84,     4,   408,     9,     4,  7696,
           13,     4,    21,     6,    12,    11,     8,   397,    67,    20,
          440,     5,     7,    14,   520,    12,     6,   102,     5,     4,
           24,   188,    76,  3356,  1174,  2433,  6288,   221,   116,     6,
          261,     8,   123,  3814,     5,   161,  1900,  3565,     0,  4109,
           10,    80,    12,   209,   238,    29,  2089,    60,    13,    16,
         2843,     5,    29,   448,     4,    24,    31,   121,  3394,    10,
         2433,     5,  3173,  5852,  4531,   306,     4,   704, 